In [ ]:
import torchvision
import numpy as np
import matplotlib
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger



In [ ]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False)

In [ ]:
import numpy as np
from src import preprocess

validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1)
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_41043/2385522803.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_train = np.array(train_dataset.targets[:train_size]).reshape(train_size,1)
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_41043/2385522803.py:16: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_valid = np.array(train_dataset.targets[train_size:]).reshape(validation_size,1)
/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/

<class 'torch.Tensor'> <class 'numpy.ndarray'>
torch.Size([48000, 784]) (48000, 10)


In [ ]:
activation_func = activation_functions.ReLU()
loss_func = loss_functions.CrossEntropyLoss()
optimizer = optimizers.SGD()
W_initializers = initializers.HeInitializer()
B_initializer = initializers.ConstantInitializer(0)
input_size = x_train.shape[1]
hidden_size = 392
output_size = 10

Input_layer = layer.Layer(input_size=input_size, output_size=output_size, activation_function=activation_func, weight_initializer=W_initializers, bias_initializer=B_initializer)
layers = [Input_layer]
FeedForward = FFNN.FFNN(layers=layers, loss_function=loss_func, optimizer=optimizer)


In [ ]:
epochs = 10
batch_size = 32
training.Trainer(FeedForward, loss_function=loss_func, optimizer=optimizer).train(X=x_train, y=y_train, epochs = epochs, batch_size= batch_size,shuffle=False)



TypeError: SGD.step() missing 1 required positional argument: 'gradient'